<a href="https://colab.research.google.com/github/Kuzay3t/Augumented-Dataset-with-SMOTE-NN/blob/main/Pipeline_leakge_location_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Import necessary libraries
We'll need `sklearn` for the model and data, `pandas` for data handling (optional, but good practice), and `matplotlib` for visualization (optional).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

### 2. Load Data


In [ ]:
from google.colab import files
files = files.upload()

Saving augmented_pipeline_data.csv to augmented_pipeline_data.csv


In [ ]:
df = pd.read_csv("augmented_pipeline_data.csv")

In [ ]:
df.head()

,Group_No,Leak_Location_1,Leak_Location_2,Inflow_Rate,Pressure_Drop,Inflow_Velocity,Outlet_Pressure
0,1,0.4,0.75,0.000313,1955.0,1.26,15340.0
1,2,0.4,0.75,0.000323,2028.0,1.30,13448.0
2,3,0.4,0.75,0.000366,2657.0,1.47,5367.0
3,4,0.6,0.86,0.000321,2112.0,1.30,13244.0
4,5,0.6,0.86,0.000344,2450.0,1.39,9612.0


In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

print("Features (X) head:")
display(X.head())
print("Target (y) unique values:")
print(pd.Series(y).unique())

Features (X) head:


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


Target (y) unique values:
[0 1 2]


### 3. Split the Data into Training and Testing Sets
It's crucial to split your data to evaluate the model's performance on unseen data. We'll use 80% for training and 20% for testing.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")

### 4. Initialize and Train the Random Forest Classifier
Create an instance of `RandomForestClassifier` and train it on the training data. Key parameters include `n_estimators` (number of trees) and `random_state` for reproducibility.

In [ ]:
# Initialize the Random Forest Classifier
# n_estimators: The number of trees in the forest.
# random_state: Controls both the randomness of the bootstrapping of the samples used when building trees
# and the sampling of the features to consider when looking for the best split at each node.
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Train the model
rf_classifier.fit(X_train, y_train)

print("Random Forest Classifier trained successfully!")

### 5. Make Predictions on the Test Set
Once the model is trained, use it to predict the target variable for the test set.

In [ ]:
y_pred = rf_classifier.predict(X_test)

print("First 5 predictions:", y_pred[:5])
print("First 5 actual values:", y_test[:5])

### 6. Evaluate the Model's Performance
Evaluate how well the model performed using metrics like accuracy, classification report (precision, recall, f1-score), and a confusion matrix.

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Display classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

# Display confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

### 7. Feature Importance (Optional)
Random Forest models can also provide insights into which features were most important for making predictions.

In [ ]:
feature_importances = pd.Series(rf_classifier.feature_importances_, index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importances.values, y=feature_importances.index, palette='viridis')
plt.title('Feature Importances')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.show()

display(feature_importances)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import interp1d
import warnings
warnings.filterwarnings('ignore')

# Load the original dataset
data = {
    'Group_No': range(1, 31),
    'Leak_Location_1': [0.4, 0.4, 0.4, 0.6, 0.6, 0.6, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14,
                        0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.33, 0.33, 0.33, 0.53, 0.53, 0.53,
                        0.25, 0.25, 0.25, 0.4, 0.4, 0.4],
    'Leak_Location_2': [0.75, 0.75, 0.75, 0.86, 0.86, 0.86, 0.6, 0.6, 0.6, 0.39, 0.39, 0.39,
                        0.6, 0.6, 0.6, 0.75, 0.75, 0.75, 0.66, 0.66, 0.66, 0.75, 0.75, 0.75,
                        0.47, 0.47, 0.47, 0.86, 0.86, 0.86],
    'Inflow_Rate': [3.13e-4, 3.23e-4, 3.66e-4, 3.21e-4, 3.44e-4, 3.58e-4, 3.20e-4, 3.52e-4, 3.70e-4,
                    3.25e-4, 3.52e-4, 3.64e-4, 3.13e-4, 3.28e-4, 3.62e-4, 3.27e-4, 3.41e-4, 3.64e-4,
                    3.26e-4, 3.47e-4, 3.61e-4, 3.24e-4, 3.50e-4, 3.69e-4, 3.23e-4, 3.51e-4, 3.67e-4,
                    3.30e-4, 3.48e-4, 3.71e-4],
    'Pressure_Drop': [1955, 2028, 2657, 2112, 2450, 2509, 2141, 2571, 2770, 2176, 2504, 2699,
                     2083, 2293, 2698, 2221, 2365, 2632, 2215, 2490, 2611, 2167, 2478, 2749,
                     2194, 2501, 2898, 2195, 2400, 2755],
    'Inflow_Velocity': [1.26, 1.3, 1.47, 1.3, 1.39, 1.44, 1.29, 1.42, 1.49, 1.31, 1.42, 1.46,
                       1.26, 1.32, 1.46, 1.32, 1.37, 1.47, 1.32, 1.4, 1.47, 1.3, 1.41, 1.49,
                       1.3, 1.41, 1.48, 1.33, 1.4, 1.5],
    'Outlet_Pressure': [15340, 13448, 5367, 13244, 9612, 6580, 14792, 9068, 5891, 13758, 9142, 6382,
                       14854, 12724, 6088, 13213, 10136, 5987, 12908, 9334, 6021, 14769, 9267, 6048,
                       14737, 9521, 6106, 13708, 9073, 5108]
}

df_original = pd.DataFrame(data)

print("Original Dataset Shape:", df_original.shape)
print("\nOriginal Data Statistics:")
print(df_original.describe())

# Augmentation Strategy
class PipelineDataAugmenter:
    def __init__(self, df, target_samples=300):
        self.df = df.copy()
        self.target_samples = target_samples
        self.feature_cols = ['Inflow_Rate', 'Pressure_Drop', 'Inflow_Velocity', 'Outlet_Pressure']
        self.target_cols = ['Leak_Location_1', 'Leak_Location_2']

    def gaussian_noise_augmentation(self, n_samples_per_original=3):
        """Add Gaussian noise to create variations"""
        augmented_samples = []

        for idx, row in self.df.iterrows():
            for _ in range(n_samples_per_original):
                new_row = row.copy()

                # Add noise with varying intensity based on feature characteristics
                new_row['Inflow_Rate'] += np.random.normal(0, 2e-6)
                new_row['Pressure_Drop'] += np.random.normal(0, 50)
                new_row['Inflow_Velocity'] += np.random.normal(0, 0.02)
                new_row['Outlet_Pressure'] += np.random.normal(0, 200)

                # Small noise to leak locations (targets)
                new_row['Leak_Location_1'] += np.random.normal(0, 0.01)
                new_row['Leak_Location_2'] += np.random.normal(0, 0.01)

                # Clip to reasonable ranges
                new_row['Leak_Location_1'] = np.clip(new_row['Leak_Location_1'], 0, 1)
                new_row['Leak_Location_2'] = np.clip(new_row['Leak_Location_2'], 0, 1)
                new_row['Inflow_Rate'] = np.clip(new_row['Inflow_Rate'], 2e-4, 5e-4)
                new_row['Inflow_Velocity'] = np.clip(new_row['Inflow_Velocity'], 1.0, 2.0)

                augmented_samples.append(new_row)

        return pd.DataFrame(augmented_samples)

    def interpolation_augmentation(self, n_samples=50):
        """Create samples by interpolating between similar existing samples"""
        augmented_samples = []

        # Group by similar leak locations
        unique_locations = self.df.groupby(['Leak_Location_1', 'Leak_Location_2']).groups

        for location, indices in unique_locations.items():
            if len(indices) >= 2:
                samples = self.df.loc[list(indices)]

                # Create interpolated samples
                for i in range(len(samples) - 1):
                    for alpha in np.linspace(0.2, 0.8, 3):
                        new_row = {}
                        row1 = samples.iloc[i]
                        row2 = samples.iloc[i + 1]

                        # Interpolate all features
                        for col in self.df.columns:
                            if col != 'Group_No':
                                new_row[col] = alpha * row1[col] + (1 - alpha) * row2[col]

                        augmented_samples.append(new_row)

        return pd.DataFrame(augmented_samples[:n_samples])

    def physics_based_augmentation(self, n_samples=50):
        """Generate samples based on physical relationships"""
        augmented_samples = []

        for _ in range(n_samples):
            # Randomly select a base sample
            base_idx = np.random.randint(0, len(self.df))
            base_row = self.df.iloc[base_idx].copy()

            new_row = base_row.copy()

            # Apply physics-based modifications
            # Higher inflow rate -> higher pressure drop (generally)
            rate_multiplier = np.random.uniform(0.9, 1.1)
            new_row['Inflow_Rate'] *= rate_multiplier
            new_row['Pressure_Drop'] *= (rate_multiplier ** 0.8) * np.random.uniform(0.95, 1.05)

            # Velocity and flow rate correlation
            new_row['Inflow_Velocity'] *= rate_multiplier * np.random.uniform(0.98, 1.02)

            # Outlet pressure inversely related to pressure drop
            pressure_factor = 1 - (rate_multiplier - 1) * 0.5
            new_row['Outlet_Pressure'] *= pressure_factor * np.random.uniform(0.95, 1.05)

            # Slight variation in leak locations
            new_row['Leak_Location_1'] += np.random.normal(0, 0.02)
            new_row['Leak_Location_2'] += np.random.normal(0, 0.02)

            # Clip values
            new_row['Leak_Location_1'] = np.clip(new_row['Leak_Location_1'], 0.1, 0.9)
            new_row['Leak_Location_2'] = np.clip(new_row['Leak_Location_2'], 0.3, 0.95)
            new_row['Inflow_Rate'] = np.clip(new_row['Inflow_Rate'], 2.5e-4, 4e-4)
            new_row['Inflow_Velocity'] = np.clip(new_row['Inflow_Velocity'], 1.2, 1.55)
            new_row['Outlet_Pressure'] = np.clip(new_row['Outlet_Pressure'], 4000, 16000)
            new_row['Pressure_Drop'] = np.clip(new_row['Pressure_Drop'], 1800, 3000)

            augmented_samples.append(new_row)

        return pd.DataFrame(augmented_samples)

    def smote_like_augmentation(self, n_samples=50):
        """SMOTE-inspired augmentation for continuous targets"""
        augmented_samples = []

        for _ in range(n_samples):
            # Pick two random samples
            idx1, idx2 = np.random.choice(len(self.df), 2, replace=False)
            sample1 = self.df.iloc[idx1]
            sample2 = self.df.iloc[idx2]

            # Generate random interpolation factor
            lambda_val = np.random.uniform(0.3, 0.7)

            new_row = {}
            for col in self.df.columns:
                if col != 'Group_No':
                    new_row[col] = lambda_val * sample1[col] + (1 - lambda_val) * sample2[col]

            augmented_samples.append(new_row)

        return pd.DataFrame(augmented_samples)

    def augment(self):
        """Apply all augmentation techniques"""
        print("\n=== Augmentation Process ===")

        # Apply different augmentation techniques
        print("1. Applying Gaussian Noise Augmentation...")
        aug1 = self.gaussian_noise_augmentation(n_samples_per_original=3)
        print(f"   Generated {len(aug1)} samples")

        print("2. Applying Interpolation Augmentation...")
        aug2 = self.interpolation_augmentation(n_samples=50)
        print(f"   Generated {len(aug2)} samples")

        print("3. Applying Physics-Based Augmentation...")
        aug3 = self.physics_based_augmentation(n_samples=60)
        print(f"   Generated {len(aug3)} samples")

        print("4. Applying SMOTE-Like Augmentation...")
        aug4 = self.smote_like_augmentation(n_samples=50)
        print(f"   Generated {len(aug4)} samples")

        # Combine all augmented data with original
        augmented_df = pd.concat([self.df, aug1, aug2, aug3, aug4], ignore_index=True)

        # Reset group numbers
        augmented_df['Group_No'] = range(1, len(augmented_df) + 1)

        print(f"\n=== Augmentation Complete ===")
        print(f"Original samples: {len(self.df)}")
        print(f"Augmented samples: {len(augmented_df)}")
        print(f"Increase factor: {len(augmented_df) / len(self.df):.2f}x")

        return augmented_df

# Perform augmentation
augmenter = PipelineDataAugmenter(df_original, target_samples=300)
df_augmented = augmenter.augment()

print("\n=== Augmented Dataset Statistics ===")
print(df_augmented.describe())

# Save augmented dataset
df_augmented.to_csv('augmented_pipeline_data.csv', index=False)
print("\n✓ Augmented dataset saved as 'augmented_pipeline_data.csv'")

# Visualize distributions
print("\n=== Feature Distribution Comparison ===")
print("\nOriginal Data Ranges:")
for col in ['Leak_Location_1', 'Leak_Location_2', 'Inflow_Rate', 'Pressure_Drop',
            'Inflow_Velocity', 'Outlet_Pressure']:
    print(f"{col}: [{df_original[col].min():.6f}, {df_original[col].max():.6f}]")

print("\nAugmented Data Ranges:")
for col in ['Leak_Location_1', 'Leak_Location_2', 'Inflow_Rate', 'Pressure_Drop',
            'Inflow_Velocity', 'Outlet_Pressure']:
    print(f"{col}: [{df_augmented[col].min():.6f}, {df_augmented[col].max():.6f}]")

print("\n=== Sample of Augmented Data ===")
print(df_augmented.head(10))
print("\n...")
print(df_augmented.tail(10))

Original Dataset Shape: (30, 7)

Original Data Statistics:
        Group_No  Leak_Location_1  Leak_Location_2  Inflow_Rate  \
count  30.000000        30.000000        30.000000    30.000000   
mean   15.500000         0.329000         0.669000     0.000344   
std     8.803408         0.148819         0.150753     0.000019   
min     1.000000         0.140000         0.390000     0.000313   
25%     8.250000         0.250000         0.600000     0.000325   
50%    15.500000         0.290000         0.705000     0.000347   
75%    22.750000         0.400000         0.750000     0.000362   
max    30.000000         0.600000         0.860000     0.000371   

       Pressure_Drop  Inflow_Velocity  Outlet_Pressure  
count      30.000000        30.000000        30.000000  
mean     2417.233333         1.385333     10074.200000  
std       255.965472         0.077714      3550.004919  
min      1955.000000         1.260000      5108.000000  
25%      2194.250000         1.312500      6175.0000

In [3]:
# import dataset
import pandas as pd
import numpy as np
from google.colab import files
file = files.upload()

Saving Main Dataset.csv to Main Dataset (1).csv


In [4]:
main_df = pd.read_csv("Main Dataset.csv", dtype=str, encoding="utf-8", engine="python")
print(main_df.columns)


Index(['Grouo No', 'Leak Location 1', 'Leak Location 2', 'Inflow Rate (m³/s)',
       'Pressure Drop (Pa)', 'Inflow velocity (m/s)', 'Outlet pressure (Pa)'],
      dtype='object')


In [5]:
print(main_df["Inflow Rate (m³/s)"].head(10).tolist())


['3.13 ×10⁻⁴', '3.23 ×10⁻⁴', '3.66 ×10⁻⁴', '3.21 ×10⁻⁴', '3.44 ×10⁻⁴', '3.58 ×10⁻⁴', '3.20 ×10⁻⁴', '3.52 ×10⁻⁴', '3.70 ×10⁻⁴', '3.25 ×10⁻⁴']


In [6]:
for i,val in enumerate(main_df["Inflow Rate (m³/s)"].head(10)):
    print(i, repr(val))


0 '3.13 ×10⁻⁴'
1 '3.23 ×10⁻⁴'
2 '3.66 ×10⁻⁴'
3 '3.21 ×10⁻⁴'
4 '3.44 ×10⁻⁴'
5 '3.58 ×10⁻⁴'
6 '3.20 ×10⁻⁴'
7 '3.52 ×10⁻⁴'
8 '3.70 ×10⁻⁴'
9 '3.25 ×10⁻⁴'


In [7]:
import re
import numpy as np

def parse_inflow_rate(val):
    """
    Convert strings like '3.13 ×10⁻⁴' into numeric floats like 3.13e-4
    """
    val = str(val).strip()

    # 1. Replace unicode multiplication sign with " * "
    val = val.replace("×", "*")

    # 2. Replace superscript minus and superscript digits
    val = val.replace("⁻", "-")   # superscript minus → normal minus
    val = val.replace("⁰", "0")
    val = val.replace("¹", "1")
    val = val.replace("²", "2")
    val = val.replace("³", "3")
    val = val.replace("⁴", "4")
    val = val.replace("⁵", "5")
    val = val.replace("⁶", "6")
    val = val.replace("⁷", "7")
    val = val.replace("⁸", "8")
    val = val.replace("⁹", "9")

    # After replacements, string looks like: "3.13 *10-4"

    # 3. Replace "*10-4" with "e-4"
    val = re.sub(r"(\d+\.?\d*)\s*\*\s*10(-?\d+)", r"\1e\2", val)

    # 4. Convert to float
    try:
        return float(val)
    except:
        return np.nan


In [8]:
main_df["Inflow_Rate"] = main_df["Inflow Rate (m³/s)"].apply(parse_inflow_rate)


In [9]:
print(main_df["Inflow_Rate"].head(10))


0    0.000313
1    0.000323
2    0.000366
3    0.000321
4    0.000344
5    0.000358
6    0.000320
7    0.000352
8    0.000370
9    0.000325
Name: Inflow_Rate, dtype: float64


In [10]:
def clean_number(val):
    val = str(val).replace(",", "").strip()
    try:
        return float(val)
    except:
        return np.nan

main_df["Pressure_Drop"] = main_df["Pressure Drop (Pa)"].apply(clean_number)
main_df["Inflow_Velocity"] = main_df["Inflow velocity (m/s)"].apply(clean_number)
main_df["Outlet_Pressure"] = main_df["Outlet pressure (Pa)"].apply(clean_number)


In [11]:
print(main_df.isna().sum())
print(main_df.head())
print(main_df.dtypes)


Grouo No                 0
Leak Location 1          0
Leak Location 2          0
Inflow Rate (m³/s)       0
Pressure Drop (Pa)       0
Inflow velocity (m/s)    0
Outlet pressure (Pa)     0
Inflow_Rate              0
Pressure_Drop            0
Inflow_Velocity          0
Outlet_Pressure          0
dtype: int64
  Grouo No Leak Location 1 Leak Location 2 Inflow Rate (m³/s)  \
0        1             0.4            0.75         3.13 ×10⁻⁴   
1        2             0.4            0.75         3.23 ×10⁻⁴   
2        3             0.4            0.75         3.66 ×10⁻⁴   
3        4             0.6            0.86         3.21 ×10⁻⁴   
4        5             0.6            0.86         3.44 ×10⁻⁴   

  Pressure Drop (Pa) Inflow velocity (m/s) Outlet pressure (Pa)  Inflow_Rate  \
0              1,955                  1.26               15,340     0.000313   
1              2,028                   1.3               13,448     0.000323   
2              2,657                  1.47                5

In [12]:
# Drop old string columns now that numeric versions exist
main_df_clean = main_df[[
    "Leak Location 1",
    "Leak Location 2",
    "Inflow_Rate",
    "Pressure_Drop",
    "Inflow_Velocity",
    "Outlet_Pressure"
]].copy()

# Rename to clean ML-ready names
main_df_clean.columns = [
    "Leak_Location_1",
    "Leak_Location_2",
    "Inflow_Rate",
    "Pressure_Drop",
    "Inflow_Velocity",
    "Outlet_Pressure"
]

print(main_df_clean.head())


  Leak_Location_1 Leak_Location_2  Inflow_Rate  Pressure_Drop  \
0             0.4            0.75     0.000313         1955.0   
1             0.4            0.75     0.000323         2028.0   
2             0.4            0.75     0.000366         2657.0   
3             0.6            0.86     0.000321         2112.0   
4             0.6            0.86     0.000344         2450.0   

   Inflow_Velocity  Outlet_Pressure  
0             1.26          15340.0  
1             1.30          13448.0  
2             1.47           5367.0  
3             1.30          13244.0  
4             1.39           9612.0  


In [13]:
# Compute baseline pressures
baseline_df = main_df_clean.groupby("Inflow_Rate")["Pressure_Drop"].mean().reset_index()
baseline_df.columns = ["Inflow_Rate", "Baseline_Pressure_Drop"]

# Merge
main_df_clean = pd.merge(main_df_clean, baseline_df, on="Inflow_Rate", how="left")

# Dimensionless real ΔP_D
main_df_clean["DeltaP_D_real"] = main_df_clean["Pressure_Drop"] / main_df_clean["Baseline_Pressure_Drop"]

print(main_df_clean.head())
print("ΔP_D Range:", main_df_clean["DeltaP_D_real"].min(), "to", main_df_clean["DeltaP_D_real"].max())


  Leak_Location_1 Leak_Location_2  Inflow_Rate  Pressure_Drop  \
0             0.4            0.75     0.000313         1955.0   
1             0.4            0.75     0.000323         2028.0   
2             0.4            0.75     0.000366         2657.0   
3             0.6            0.86     0.000321         2112.0   
4             0.6            0.86     0.000344         2450.0   

   Inflow_Velocity  Outlet_Pressure  Baseline_Pressure_Drop  DeltaP_D_real  
0             1.26          15340.0                  2019.0       0.968301  
1             1.30          13448.0                  2111.0       0.960682  
2             1.47           5367.0                  2657.0       1.000000  
3             1.30          13244.0                  2112.0       1.000000  
4             1.39           9612.0                  2450.0       1.000000  
ΔP_D Range: 0.9606821411653245 to 1.0393178588346754


In [14]:
df_for_synthesis = main_df_clean.copy()


In [15]:
import numpy as np
import pandas as pd

# ===============================================================
# STEP 0 — Use the CLEANED main_df_clean from previous steps
# ===============================================================
df = main_df_clean.copy()

# ===============================================================
# STEP 1 — Extract REAL ΔP_D range for scaling
# ===============================================================

real_min = df["DeltaP_D_real"].min()
real_max = df["DeltaP_D_real"].max()
print("Real ΔP_D Range:", real_min, "to", real_max)

# ===============================================================
# STEP 2 — CFD MODEL EQUATION from research paper
# ===============================================================

a = 23.94
b = -0.1598
c = -23.74
m = -0.4789
w = 0.0989

def CFD_equation(L1, L2):
    return (a
            + b * np.sin(m * np.pi * L1 * L2)
            + c * np.exp(-(w * L2)**2))

# ===============================================================
# STEP 3 — Generate Synthetic Leak Locations
# ===============================================================

N = 5000   # number of synthetic samples

np.random.seed(42)
L1_syn = np.random.uniform(0.10, 0.90, N)
L2_syn = np.random.uniform(0.10, 0.90, N)

# Raw CFD ΔP values (dimensionless)
DeltaP_CFD_raw = CFD_equation(L1_syn, L2_syn)

# ===============================================================
# STEP 4 — Rescale CFD ΔP_D to match REAL dataset ΔP_D range
# ===============================================================

DeltaP_norm = (DeltaP_CFD_raw - DeltaP_CFD_raw.min()) / (DeltaP_CFD_raw.max() - DeltaP_CFD_raw.min())
DeltaP_syn_D = real_min + DeltaP_norm * (real_max - real_min)

# ===============================================================
# STEP 5 — SAMPLE INFLOW RATES + FIX BASELINE MISMATCH
# ===============================================================

# Round main dataset inflow rates to avoid float mismatch errors
df["Inflow_Rate"] = df["Inflow_Rate"].round(7)

# Generate synthetic inflow rates (sample from real)
inflow_syn = np.random.choice(df["Inflow_Rate"], N)
inflow_syn = np.round(inflow_syn, 7)   # round them to match keys

# Create baseline lookup mapping
baseline_map = dict(zip(df["Inflow_Rate"], df["Baseline_Pressure_Drop"]))

# SAFE baseline assignment (avoids KeyErrors)
default_baseline = np.mean(list(baseline_map.values()))

baseline_assigned = np.array([
    baseline_map.get(rate, default_baseline)
    for rate in inflow_syn
])

# ===============================================================
# STEP 6 — Compute SYNTHETIC dimensional pressure drops
# ===============================================================

Pressure_syn = DeltaP_syn_D * baseline_assigned

# Add 2% engineering noise
Pressure_syn = Pressure_syn + np.random.normal(0, Pressure_syn * 0.02)

# ===============================================================
# STEP 7 — Sample SYNTHETIC velocities & outlet pressures
# ===============================================================

vel_syn = np.random.choice(df["Inflow_Velocity"], N)
outP_syn = np.random.choice(df["Outlet_Pressure"], N)

# ===============================================================
# STEP 8 — Build the SYNTHETIC DataFrame
# ===============================================================

synthetic_df = pd.DataFrame({
    "Leak_Location_1": L1_syn,
    "Leak_Location_2": L2_syn,
    "Inflow_Rate": inflow_syn,
    "Pressure_Drop": Pressure_syn,
    "Inflow_Velocity": vel_syn,
    "Outlet_Pressure": outP_syn
})

# ===============================================================
# STEP 9 — Create FINAL merged dataset
# ===============================================================

final_df = pd.concat([
    df[[
        "Leak_Location_1", "Leak_Location_2", "Inflow_Rate",
        "Pressure_Drop", "Inflow_Velocity", "Outlet_Pressure"
    ]],
    synthetic_df
], ignore_index=True)

final_df.to_csv("physics_augmented_dataset_FINAL.csv", index=False)

print("Synthetic dataset successfully generated!")
print("Total samples:", final_df.shape)


Real ΔP_D Range: 0.9606821411653245 to 1.0393178588346754
Synthetic dataset successfully generated!
Total samples: (5030, 6)


In [16]:
import numpy as np
import pandas as pd

# ===============================================================
# STEP 0 — Use the CLEANED main_df_clean produced earlier
# ===============================================================
df = main_df_clean.copy()

# ===============================================================
# STEP 1 — Extract REAL ΔP_D range for scaling
# ===============================================================
real_min = df["DeltaP_D_real"].min()
real_max = df["DeltaP_D_real"].max()

print("Real ΔP_D Range:", real_min, "to", real_max)

# ===============================================================
# STEP 2 — CFD MODEL EQUATION (from the paper)
# ===============================================================
a = 23.94
b = -0.1598
c = -23.74
m = -0.4789
w = 0.0989

def CFD_equation(L1, L2):
    return (a
            + b * np.sin(m * np.pi * L1 * L2)
            + c * np.exp(-(w * L2)**2))

# ===============================================================
# STEP 3 — Generate Synthetic Leak Locations
# ===============================================================
N = 5000  # Number of synthetic samples

np.random.seed(42)
L1_syn = np.random.uniform(0.10, 0.90, N)
L2_syn = np.random.uniform(0.10, 0.90, N)

# Raw CFD ΔP_D (dimensionless)
DeltaP_CFD_raw = CFD_equation(L1_syn, L2_syn)

# ===============================================================
# STEP 4 — Rescale CFD ΔP_D to match REAL ΔP_D range
# ===============================================================
DeltaP_norm = (DeltaP_CFD_raw - DeltaP_CFD_raw.min()) / (DeltaP_CFD_raw.max() - DeltaP_CFD_raw.min())
DeltaP_syn_D = real_min + DeltaP_norm * (real_max - real_min)

# ===============================================================
# STEP 5 — USE STRING KEY MAPPING FOR BASELINE PRESSURES (FIX)
# ===============================================================

# Convert inflow rates to STRING keys
df["Inflow_Rate_Key"] = df["Inflow_Rate"].apply(lambda x: f"{x:.10f}")

# Build baseline map with string keys
baseline_map = dict(zip(df["Inflow_Rate_Key"], df["Baseline_Pressure_Drop"]))

# Sample synthetic inflow rates using string keys
inflow_syn_keys = np.random.choice(df["Inflow_Rate_Key"], N)

# Baseline assignment using string-keys (GUARANTEED MATCH)
baseline_assigned = np.array([baseline_map[key] for key in inflow_syn_keys])

# Convert keys back to float for final dataset
inflow_syn = np.array([float(key) for key in inflow_syn_keys])

# ===============================================================
# STEP 6 — Compute SYNTHETIC dimensional pressure drops
# ===============================================================
Pressure_syn = DeltaP_syn_D * baseline_assigned

# Add 2% engineering noise
Pressure_syn = Pressure_syn + np.random.normal(0, Pressure_syn * 0.02)

# ===============================================================
# STEP 7 — Sample realistic velocities & outlet pressures
# ===============================================================
vel_syn  = np.random.choice(df["Inflow_Velocity"], N)
outP_syn = np.random.choice(df["Outlet_Pressure"], N)

# ===============================================================
# STEP 8 — Build the SYNTHETIC DATAFRAME
# ===============================================================
synthetic_df = pd.DataFrame({
    "Leak_Location_1": L1_syn,
    "Leak_Location_2": L2_syn,
    "Inflow_Rate": inflow_syn,
    "Pressure_Drop": Pressure_syn,
    "Inflow_Velocity": vel_syn,
    "Outlet_Pressure": outP_syn
})

# ===============================================================
# STEP 9 — MERGE REAL + SYNTHETIC (FINAL DATA)
# ===============================================================
final_df = pd.concat([
    df[[
        "Leak_Location_1",
        "Leak_Location_2",
        "Inflow_Rate",
        "Pressure_Drop",
        "Inflow_Velocity",
        "Outlet_Pressure"
    ]],
    synthetic_df
], ignore_index=True)

# ===============================================================
# STEP 10 — SAVE FINAL DATASET
# ===============================================================
final_df.to_csv("physics_augmented_dataset_FINAL_FINAL.csv", index=False)

print("\nSynthetic dataset successfully generated!")
print("Total samples:", final_df.shape)


Real ΔP_D Range: 0.9606821411653245 to 1.0393178588346754

Synthetic dataset successfully generated!
Total samples: (5030, 6)
